# Polynominal regression

## Task

- Load dataset with sizes and prices
- Explore and display the data
- Create a linear model and find out its R2
- Create polynomial models with different degrees, find their R2
- Select the best model 

# Data loading

In [1]:
import pandas as pd 

In [2]:
data = pd.read_csv ("..\dataset\prices.csv",  sep=';')

# Linear model
- standardize the data
- create and train a linear model
- display model parameters
- visualize the linear model
- display model scores

# Create a polynomial regression model
- create a polynomial 1D model with steps 2 - 8
- display models
- display their parameters
- show their scores
- Which model would you choose?